In [1]:
import pandas as pd
import numpy as np


import matplotlib.pylab as plt
import seaborn as sns
sns.set_style('whitegrid')

from collections import defaultdict
from progressbar import ProgressBar
import re
from multiprocessing import Pool
import time

import scipy
import networkx as nx
import re

import os
import random
import statsmodels.api as sm
from collections import Counter

from statsmodels.stats.multitest import multipletests

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

/home/italodovalle/.local/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [6]:
infolder = os.getcwd()[:-15] + 'data/'
outfolder = os.getcwd()[:-15] + 'output/'
dbs = os.getcwd()[:-15] + 'data/databases/'

In [7]:
os.chdir(os.getcwd()[:-15] + 'python/')
import guney_code.wrappers as wrappers

from guney_code.wrappers import network_utilities, get_random_nodes, calculate_closest_distance, calculate_separation_proximity, calculate_proximity

import toolbox.network_utils as network_utils

GenRev not found, steiner wont work
Import error: Negex. Using keyword matching instead
Import error: Funcassociate. Make sure that funcassociate is in toolbox!


In [8]:
infile = infolder + 'Human_Interactome_201706.txt'
header=True
sep = '\t'
columns = ['#Protein A', 'Protein B']
lcc = False
G = network_utils.parse_interactome(infile, sep, header, columns, lcc=True)

print (len(G.nodes()), 'nodes')
print (len(G.edges()), 'edges')

17651 nodes
351393 edges


In [9]:
polyphenol = pd.read_csv(infolder + 'PhenolExplorer_CTD-Stitch.csv',index_col = 0)
polyphenol = polyphenol[polyphenol.experimental > 0]
chemical2genes = defaultdict(list)
for i in polyphenol.index:
    name = polyphenol.chemical.loc[i]
    chemical2genes[name].append(polyphenol.entrezid.loc[i])
    
for i in chemical2genes.keys():
    chemical2genes[i] = list(set(chemical2genes[i]))

In [19]:
disease2genes = {}
for i in open(infolder + 'Guney2016_GenesDisease.tsv').readlines():
    v = i.rstrip().split('\t')
    disease = v[1]
    genes = v[2:]
    if len(genes) > 19:
        disease2genes[disease] = [int(i) for i in genes]

## Import GCT

* Perturbation signatures (in z-scores) of MCF7 cells treated with polyphenols were downloaded from the CLUE.io website (https://clue.io/)

In [14]:
def parse_gct(infile):
    
    dt = pd.read_table(infile, sep = '\t', skiprows=2)
    cols = list(dt.columns[9:])
    desc = dt.loc[:7, cols]
    cols.append('id')
    exp_matrix = dt[cols]
    exp_matrix = dt.iloc[8:]
    attributes = defaultdict(dict)
    desc = dt.loc[:7, cols]
    for i in desc.columns[:-1]:
        for j in desc.index:
            attributes[i][desc['id'].loc[j]] = desc[i].loc[j]
    attributes = pd.DataFrame.from_dict(attributes, orient='index').reset_index()
    attributes['distil_cc_q75'] = attributes['distil_cc_q75'].astype(float)
    return(exp_matrix, attributes)


def get_sp(gene):
    if gene in G.nodes():
        s = network_utils.calculate_closest_distance(G,[gene], genes)
    else:
        s = float('nan')
    t = (gene,s)
    return(t)

In [18]:
def get_gsea(geneset, nongeneset, df,dataframe=False):
    dic = {}
    N_H = len(geneset)
    N = df.shape[0]
    n_r = np.sum(df['value'][df.gene.isin(geneset)].abs())
    for rank in range(N):
        dx = df.iloc[:rank]
        s = list(set(geneset) & (set(dx['gene'])))
        p_hit = np.sum(dx[dx.gene.isin(s)]['value'].abs()/n_r)
        p_miss = 1.0/(N - N_H) * dx[~dx.gene.isin(geneset)].shape[0]
        dic[rank] = p_hit - p_miss
    dic = pd.DataFrame.from_dict(dic, orient='index')
    if dataframe:
        return(dic)
    else:
        return(dic[0].abs().max())
    
    
def get_gsea_performance(geneset, nongeneset, df):
    df = df.reset_index()
    ranks_mapped = list(df[df.gene.isin(geneset)].index)
    ranks = []
    for i in ranks_mapped:
        ranks.append(i - 1)
        ranks.append(i)
        ranks.append(i + 1)
    dic = {}
    N_H = len(geneset)
    N = df.shape[0]
    n_r = np.sum(df['value'][df.gene.isin(geneset)].abs())
    for rank in ranks:
        dx = df.iloc[:rank]
        s = list(set(geneset) & (set(dx['gene'])))
        p_hit = np.sum(dx[dx.gene.isin(s)]['value'].abs()/n_r)
        p_miss = 1.0/(N - N_H) * dx[~dx.gene.isin(geneset)].shape[0]
        dic[rank] = p_hit - p_miss
    dic = pd.DataFrame.from_dict(dic, orient='index')
    return(dic[0].abs().max())


def get_null(input):
    n, df = input
    geneset = df['gene'].sample(n)
    nongeneset = list(set(df['gene']) - set(geneset))
    #es = get_gsea(geneset, nongeneset, df)
    es = get_gsea_performance(geneset, nongeneset, df)
    return(es)

In [17]:
molecules = ['coumarin.gct',
 'nobiletin.gct',
 'prunetin.gct',
 'coumestrol.gct',
 'pterostilbene.gct',
 'isoliquiritigenin.gct',
 'caffeic_acid.gct',
 'apigenin.gct',
 'naringenin.gct',
 '(-)-epicatechin 3-o-gallate.gct',
 '(-)-epigallocatechin 3-o-gallate.gct',
 'naringin.gct',
 'piceatannol.gct',
 'epicatechin.gct',
 'daidzein.gct',
 'umbelliferone.gct',
 'rosmarinic acid.gtc']

In [24]:
infiles = [i for i in os.listdir(infolder + 'databases/clue/') if i.endswith('.gct')]

In [ ]:
ncpu = 1
npermutations = 10
for infile in infiles:
    
    cell_line = 'MCF7'
    
    ## parse gct file
    dt, attributes = parse_gct(infolder + 'databases/clue/' + infile)
    
    ## select experimental instances to run gsea
    selected = attributes[(attributes.cell_id == cell_line)]
    timepoints = ['24 h', '6 h']
    treats = list(set(attributes['pert_idose']))
    samples = []
    for tp in timepoints:
        for treat in treats:
            buf = selected[(selected['pert_itime'] == tp) & (selected['pert_idose'] == treat)]
            if buf.shape[0] == 1:
                samples.append(buf['index'].iloc[0])
            if buf.shape[0] > 1:
                sample = buf[buf['distil_cc_q75'] == buf['distil_cc_q75'].max()]['index'].iloc[0]
                samples.append(sample)
                
    ## prepare tables
    cols = ['id'] + samples
    data = dt[cols]
    data['id'] = data['id'].astype(int)
    data = data[data['id'].isin(list(G.nodes()))]
    for col in samples:
        data[col] = data[col].astype(float)
    
    
    ## run gsea for each experimental instance
    for test in samples:
        
        dquery = data[[test, 'id']]
        dquery = dquery.sort_values(by = test)
        dquery.columns = ['value', 'gene']
        
        dfs = []
        pbar = ProgressBar()
        dic_null = {}
        for disease in pbar(disease2genes.keys()):
            res = defaultdict(dict)
            c = 0
            geneset = list(set(disease2genes[disease]) & set(G.nodes()))
            ns = list(set(dquery['gene']) - set(geneset))
            es = get_gsea_performance(geneset, ns, dquery)
            if not len(geneset) in dic_null.keys():
                p = Pool(ncpu)
                randomsets = np.repeat(len(geneset),npermutations)
                randomsets = [[i,dquery] for i in randomsets]
                null = p.map(get_null, randomsets)
                p.close()
                dic_null[len(geneset)] = null
            res[c]['disease'] = disease
            res[c]['ES'] = es
            res[c]['Z_ES'] = (es - np.mean(dic_null[len(geneset)]))/np.std(dic_null[len(geneset)])
            res[c]['p_empirical'] = len([i for i in dic_null[len(geneset)] if i > es])/len(dic_null[len(geneset)])
            res = pd.DataFrame.from_dict(res, orient='index')
            #res.to_csv(outfolder + 'tables/tmp/%s.csv'%disease)
            dfs.append(res)
        res = pd.concat(dfs)
    
    break

## Organize Output

In [ ]:
molecule = 'genistein'
cell_line = 'MCF7'
indir = '/home/italodovalle/flavonoids/output/tables/tmp/'
dfs = []
for i in os.listdir(indir):
    if 'gsea_%s_%s'%(molecule, cell_line.lower()) in i:
        d = pd.read_csv(indir + i, index_col = 0)
        if '_' in molecule:
            v = i.split('_')
            cell, concentration, time = v[-3],v[-2],v[-1]
            chemical = molecule
        else:
            buf, chemical, cell,concentration, time =  i.split('_')
        d['cell'] = cell
        d['concentration'] = concentration
        d['duration'] = time.split('.')[0]
        d['pvalue_adj'] = multipletests(d['p_empirical'], alpha=0.05, method='fdr_bh')[1]
        dfs.append(d)
df = pd.concat(dfs)
df.to_csv(outfolder + 'tables/gsea_cmap/%s_%s_cmap_gsea.csv'%(molecule,cell_line.lower()))